In [16]:
import yaml
from access import *

In [17]:
url_cam_given_account = "database-mm2503.cgrre17yxw11.eu-west-2.rds.amazonaws.com"
database_details = {"url": url_cam_given_account, 
                    "port": 3306}
with open("credentials.yaml") as file:
  credentials = yaml.safe_load(file)
username = credentials["username"]
password = credentials["password"]
url = database_details["url"]

In [22]:
conn = create_connection(user=credentials["username"], 
                         password=credentials["password"], 
                         host=database_details["url"],
                         database="household_prices")
table_name = "pp_data"
sample_table_name = "pp_data_sample"

In [25]:
def drop_simple_db(conn: pymysql.connections.Connection):
    cur = conn.cursor()
    cur.execute("""
        DROP TABLE IF EXISTS `simple_db`
    """)

In [26]:
drop_simple_db(conn)

In [29]:
def create_simple_db(conn: pymysql.connections.Connection):
    cur = conn.cursor()
    cur.execute("""
        CREATE TABLE IF NOT EXISTS `simple_db` (
        `region` tinytext COLLATE utf8_bin NOT NULL,
        `population` bigint(20) unsigned NOT NULL
        ) DEFAULT CHARSET=utf8 COLLATE=utf8_bin;
    """)

In [30]:
create_simple_db(conn)

In [32]:
upload_csv_to_aws(conn, "simple_db", "simple_db.csv")

5

In [11]:
print(conn)
conn.ping()

In [27]:
cur = conn.cursor()
#cur.execute("USE `household_prices`")
cur.execute("show tables")
result = cur.fetchall()

In [28]:
print(result)

(('pp_data',), ('pp_data_sample',))


In [39]:
cur = conn.cursor()
cur.execute("SELECT COUNT(*) from pp_data_sample")
result = cur.fetchall()

In [40]:
print(result)

((0,),)


In [5]:
select_top(conn, table_name)

KeyboardInterrupt: 

In [41]:
create_table_for_priced_paid_data(conn, "pp_data")
#create_table_for_priced_paid_data(conn, "pp_data_sample")

KeyboardInterrupt: 

In [ ]:
num_fetched_rows, num_uploaded_rows = upload_datasets(conn, "pp_data", "pp_data_sample", 1995, 1996)

In [ ]:
print(num_fetched_rows)
print(num_uploaded_rows)

In [5]:
total_rows = custom_simple_select(conn, "pp_data")
sample_rows = custom_simple_select(conn, "pp_data_sample")
print(f"total_rows: {total_rows}, sample_rows: {sample_rows}")

KeyboardInterrupt: 

In [ ]:
def upload_2022_dataset(conn, table_name="pp_data", sample_table_name="pp_data_sample", sample_portion=0.01):
  url = "http://prod.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2022.csv"
  df = pd.read_csv(url, header=None)
  num_fetched = len(df)
  df.to_csv("price_paid_data_year_2022.csv", header=False, index=False)
  df.sample(frac=sample_portion).to_csv("sample_price_paid_data_year_2022.csv", header=False, index=False)
  num_uploaded = upload_csv_to_aws(conn, table_name, "price_paid_data_year_2022.csv")
  upload_csv_to_aws(conn, sample_table_name, "sample_price_paid_data_year_2022.csv")
  return num_fetched, num_uploaded